<a href="https://colab.research.google.com/github/Dhia99/AskBack/blob/main/extractfacture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install git+git://github.com/jaidedai/easyocr.git

  Cloning git://github.com/jaidedai/easyocr.git to /tmp/pip-req-build-rz7330wr
  Running command git clone -q git://github.com/jaidedai/easyocr.git /tmp/pip-req-build-rz7330wr
     |████████████████████████████████| 47.6 MB 2.2 MB/s 
  Created wheel for easyocr: filename=easyocr-1.4.1-py3-none-any.whl size=63744053 sha256=96426bc5adb371c0509f291a60f9c594d56534847ae306081fc21391da6718d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-dqxlne9d/wheels/e4/17/25/2d203d4a2ecdb8b4d05e4cef1fed51634287906dfafee45701
Successfully built easyocr


In [3]:
!pip install datefinder

In [4]:
import easyocr
reader = easyocr.Reader(['fr']) 

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [5]:
result = reader.readtext("/content/drive/MyDrive/Colab Notebooks/Dhia1/factures/factures/003.jpg")

In [6]:
data_invoice=[]
for x in result:
  data_invoice.append(x[-2].strip().replace('[','').replace(']',''))
  # print(x[-2])

In [7]:
start_index= data_invoice.index("TVA")
end_index = data_invoice.index("Total H. Taxes TND")
print(start_index,end_index)

sub_data_invoice =[]
for x in data_invoice[start_index+2:end_index]:
  n = x.split("|")
  for y in n:
    sub_data_invoice.append(y.strip())




print(sub_data_invoice)

31 59
['2S529EA', 'HP 15-dw2011nk i3-1005G1', '4GB ITB 15.6 HD W1O BK', '099,000', '099,000', '7 %', '2S53OEA', 'HP 15-dw2012nk i3-1005G1', '4GB ITB 15.6 HD W1O GY', '099,000', '099,000', '7 %', 'SN:CND0256739', '0 %', 'SN:CNDO132R8D', '0 %', 'T9BSOAA', 'Sacoche HP Value Topload (15.6")', '36,000', '36,000', '19 %', 'KOB38AA', 'HP SACOCHE Mallette Value 15,6"', '36,000', '36,000', '19 %']


In [8]:
list_sub_item=[]
last_index =0
for x in range(len(sub_data_invoice)):
  if '%' in sub_data_invoice[x]:
    list_sub_item.append({"a":last_index,"b":x+1})
    last_index = x+1 
    print(sub_data_invoice[x])
  

print(list_sub_item)

7 %
7 %
0 %
0 %
19 %
19 %
[{'a': 0, 'b': 6}, {'a': 6, 'b': 12}, {'a': 12, 'b': 14}, {'a': 14, 'b': 16}, {'a': 16, 'b': 21}, {'a': 21, 'b': 26}]


In [9]:
all_result_data=[]

def clean(x):
  v = x.replace(",",'.').replace("/","").replace(" ","").replace('%','').replace(']','')
  if len(v)==0:
    v="0"
  return v

for x in list_sub_item:
  extracted_data={"ref":None,"name":None,"qty":None,"unit_price":None,"amount":None,"tva":None}
  r=sub_data_invoice[x["a"]:x["b"]]
  if len(r)==5:
    extracted_data["ref"]=r[0]
    extracted_data["name"]=r[1]
    extracted_data["unit_price"]=float(clean(r[2]))
    extracted_data["amount"]=float(clean(r[3]))
    extracted_data["tva"]=float(clean(r[4]))
  elif len(r)==2:
    extracted_data["name"]=r[0]
    extracted_data["tva"]=float(clean(r[1]))
  elif len(r)==3:
    extracted_data["name"]=r[0]+" "+r[1]
    extracted_data["tva"]=float(clean(r[2]))   
  elif len(r)>5:
    extracted_data["ref"]=r[0]
    extracted_data["name"]=r[1]+" "+r[2]
    extracted_data["unit_price"]=float(clean(r[-3]))
    extracted_data["amount"]=float(clean(r[-2]))
    extracted_data["tva"]=float(clean(r[-1]))

  if extracted_data['unit_price'] and extracted_data['amount']:
    extracted_data['qty']= int(extracted_data['unit_price']/extracted_data['amount'])

  all_result_data.append(extracted_data)

  # print(r)

In [10]:
all_result_data  

[{'amount': 99.0,
  'name': 'HP 15-dw2011nk i3-1005G1 4GB ITB 15.6 HD W1O BK',
  'qty': 1,
  'ref': '2S529EA',
  'tva': 7.0,
  'unit_price': 99.0},
 {'amount': 99.0,
  'name': 'HP 15-dw2012nk i3-1005G1 4GB ITB 15.6 HD W1O GY',
  'qty': 1,
  'ref': '2S53OEA',
  'tva': 7.0,
  'unit_price': 99.0},
 {'amount': None,
  'name': 'SN:CND0256739',
  'qty': None,
  'ref': None,
  'tva': 0.0,
  'unit_price': None},
 {'amount': None,
  'name': 'SN:CNDO132R8D',
  'qty': None,
  'ref': None,
  'tva': 0.0,
  'unit_price': None},
 {'amount': 36.0,
  'name': 'Sacoche HP Value Topload (15.6")',
  'qty': 1,
  'ref': 'T9BSOAA',
  'tva': 19.0,
  'unit_price': 36.0},
 {'amount': 36.0,
  'name': 'HP SACOCHE Mallette Value 15,6"',
  'qty': 1,
  'ref': 'KOB38AA',
  'tva': 19.0,
  'unit_price': 36.0}]

In [11]:
data_invoice

['Rue Tarek Ibn Zied',
 '4000 SOUSSE',
 'Jgozf',
 "N' téléphone",
 '73 202 788',
 "N' télécopie",
 '73 202 789',
 'SARL Au capital social de 1.270.000 DT',
 'FACTUREI BON DE LIVRAISON',
 "N'",
 'TPV-TROC-F20/2066',
 'Nom Client: STE ENNOVATUS GAMING',
 "N' doc. Ext.",
 'Adresse:',
 'AV ARAF RES BOULVARD SAHLOUL 4054',
 'Commande Clien',
 'Mat. Fiscal: 1675123S/AMIOOO',
 'Date',
 '2. novembre 2020',
 'Code Client',
 'CSC26660',
 "N' BC:",
 'TPV-TROC-C20/3185',
 'RC',
 'Paiement',
 'Réf.',
 'Désignation',
 'Qté',
 'Prix Unit.',
 'Rem.',
 'Montant',
 'TVA',
 'TPV-TROC-E2072066-',
 '2S529EA',
 'HP 15-dw2011nk i3-1005G1',
 '4GB ITB 15.6 HD W1O BK',
 '099,000',
 '099,000',
 '7 %',
 '2S53OEA',
 'HP 15-dw2012nk i3-1005G1',
 '4GB ITB 15.6 HD W1O GY',
 '099,000',
 '099,000',
 '7 %',
 'SN:CND0256739',
 '0 %',
 'SN:CNDO132R8D',
 '0 %',
 'T9BSOAA',
 'Sacoche HP Value Topload (15.6")',
 '36,000',
 '36,000',
 '19 %',
 'KOB38AA',
 'HP SACOCHE Mallette Value 15,6"',
 '36,000',
 '36,000',
 '19 %',
 'Tot

In [12]:
data_v2={"Address":None,"Mat_Fiscal":None,"Num_Phone":None}
k=0
data_v2["Address"]=data_invoice[0]+" "+data_invoice[1]
for x in data_invoice:
    
  if "Mat. Fiscal" in x:
    data_v2["Mat_Fiscal"]=x.replace("Mat. Fiscal:","").strip()

  if "téléphone" in x:
    data_v2["Num_Phone"]=data_invoice[k+1].strip()  
  
  k+=1 

data_v2


{'Address': 'Rue Tarek Ibn Zied 4000 SOUSSE',
 'Mat_Fiscal': '1675123S/AMIOOO',
 'Num_Phone': '73 202 788'}

In [13]:
import datefinder
data_v4={"Date":None}
k=0
for x in data_invoice:
  if "Date" in x:
      matches = list(datefinder.find_dates(data_invoice[k+1].replace(".","").replace("é","e")))
      if len(matches)>0:
        data_v4["Date"]=matches[0]
  k+=1 
data_v4

{'Date': datetime.datetime(2022, 11, 2, 0, 0)}

In [15]:
data_v4={"Designation":None,"Date_Ech":None,"N_Compte":None,"Banque":None}

try:
 data_v4["Designation"]=data_invoice[data_invoice.index("ticket)")+1]
except:
  pass

try:
 data_v4["Banque"]=data_invoice[data_invoice.index("Banque")+1]
except:
  pass

try:
 data_v4["N_Compte"]=data_invoice[data_invoice.index("N' Compte")+1]
except:
  data_v4["N_Compte"]=data_invoice[data_invoice.index("NP Compte")+1]
  pass

try:
 data_v4["Date_Ech"]=data_invoice[data_invoice.index("ticket)")+2]
except:
  pass

data_v4

{'Banque': 'ATB Charguia',
 'Date_Ech': '02/11/20',
 'Designation': 'ESPECE',
 'N_Compte': '01013054110300246686'}

In [16]:
import datetime


data_v3={"Total_H_Taxes":None,"Timbre_Fiscal":None,"Total_TVA":None,"Net_Payed":None,"N":None,"Date_Ech":None,"Date":None}

try:
 data_v3["Total_H_Taxes"]=float(clean(data_invoice[data_invoice.index("Total H. Taxes TND")+1]))
except:
  pass

try:  
 data_v3["Total_TVA"]=float(clean(data_invoice[data_invoice.index("Total TVA")+1]))
except:
  pass

try:  
 data_v3["Timbre_Fiscal"]=float(clean(data_invoice[data_invoice.index("Timbre Fiscal")+1]))
except:
  pass

try: 
 data_v3["Net_Payed"]=float(clean(data_invoice[data_invoice.index("Net à Payer")+1]))
except:
  pass

try:
 data_v3["Date_Ech"]=data_invoice[data_invoice.index("ticket)")+2].replace("/","-")
except:
  pass


tab_v1=[]
k=0
for x in data_invoice:
  if "TPV-" in x:
    tab_v1.append(x)

  if "Date" in x:
      matches = list(datefinder.find_dates(data_invoice[k+1].replace(".","").replace("é","e")))
      if len(matches)>0:
        data_v3["Date"]=matches[0].strftime('%m-%d-%Y') 
  k+=1   

data_v3["N"]=tab_v1[0] 





data_v3

{'Date': '11-02-2022',
 'Date_Ech': '02-11-20',
 'N': 'TPV-TROC-F20/2066',
 'Net_Payed': 2270.6,
 'Timbre_Fiscal': 0.6,
 'Total_H_Taxes': 2114.71,
 'Total_TVA': 155.29}

In [17]:
data_v1={"N":None,"Code_Client":None,"N_BC":None}

client_index= data_invoice.index("Code Client")
client_index
data_v1["Code_Client"]=data_invoice[client_index+1]


tab_v1=[]
for x in data_invoice:
  if "TPV-" in x:
    tab_v1.append(x)

data_v1["N"]=tab_v1[0] 
data_v1["N_BC"]=tab_v1[1]    

print(data_v1)


{'N': 'TPV-TROC-F20/2066', 'Code_Client': 'CSC26660', 'N_BC': 'TPV-TROC-C20/3185'}


In [18]:
# result 